In [17]:
import numpy as np
import pickle
import time

SONNET_LINES = 14
NUM_QUATRAINS = 3
QUATRAIN_LINES = 4
COUPLET_LINES = 2
NUM_SHKSP_SONNETS = 151

PUNCTUATION = [',', ':', '.', ';', '?', '!', '(', ')', '"', "'"]

SYLL_DICT = 'syll_dict.p'
STRESS_DICT = 'stress_dict.p'

############################################################
# Read in raw sonnets.
############################################################

def shksp_raw(filename='shakespeare.txt'):
    seqs = np.loadtxt(filename, delimiter='\n', dtype='str')
    return seqs

############################################################
# Different tokenizers per line.
############################################################

# simple_token1:
#   Checks back of each line for punctuation and if found, 
#   replaces with a single token of the punctuation and a 
#   newline character. Punctuation within line is attatched
#   to word on left.
def simple_token1(line):
    line = line.lower().lstrip().rstrip()
    line = line.split(' ')
    # Handle punctuation at end of line.
    last_word = list(line[-1])
    del line[-1]
    if last_word[-1] in PUNCTUATION:
        tmp = last_word[-1]
        del last_word[-1]
        line.append(''.join(last_word))
        line.append(tmp + '\n')
    else:
        line.append(''.join(last_word))
    return line

# simple_token2:
#   All punctuation are attached to word on left, no 
#   newline characters.
def simple_token2(line):
    line = line.lower().lstrip().rstrip()
    line = line.split(' ')
    return line

# simple_token3:
#   All punctuation attached to word on left. Newline
#   characters for each line.
def simple_token3(line):
    line = line.lower().lstrip().rstrip()
    line = line.split(' ')
    return line + ['\n']

# simple_token4:
#   Remove all punctuation, no newline character.
def simple_token4(line):
    line = line.lower().lstrip().rstrip()
    for punc in PUNCTUATION:
        line = line.replace(punc, '')
    line = line.split(' ')
    return line

############################################################
# Preprocess Shakespeare sonnets.
############################################################

def shksp_per_sonnet(tokenizer, filename='shakespeare.txt'):
    raw = shksp_raw(filename)
    sequences = []
    cursor = 0
    for sonnet in range(NUM_SHKSP_SONNETS):
        # Skip first line which is a number.
        cursor += 1
        # Setup sequence.
        seq = []
        for i in range(SONNET_LINES):
            seq += tokenizer(raw[cursor])
            cursor += 1
        sequences.append(seq)
    return sequences

def shksp_per_line(tokenizer, filename='shakespeare.txt'):
    raw = shksp_raw(filename)
    sequences = []
    cursor = 0
    for sonnet in range(NUM_SHKSP_SONNETS):
        # Skip first line which is a number.
        cursor += 1
        for i in range(SONNET_LINES):
            sequences.append(tokenizer(raw[cursor]))
            cursor += 1
    return sequences

def shksp_quatrains_and_couplets(tokenizer, filename='shakespeare.txt'):
    raw = shksp_raw(filename)
    quatrains = []
    couplets = []
    cursor = 0
    for sonnet in range(NUM_SHKSP_SONNETS):
        cursor += 1
        couplet = []
        for quatrain in range(NUM_QUATRAINS):
            quatrain = []
            for line in range(QUATRAIN_LINES):
                quatrain += tokenizer(raw[cursor])
                cursor += 1
            quatrains.append(quatrain)
        for line in range(COUPLET_LINES):
            couplet += tokenizer(raw[cursor])
            cursor += 1
        couplets.append(couplet)
    return quatrains, couplets

def shksp_quatrain_couplets_line(tokenizer, filename='shakespeare.txt'):
    raw = shksp_raw(filename)
    quatrains = []
    couplets = []
    cursor = 0
    for sonnet in range(NUM_SHKSP_SONNETS):
        cursor += 1
        couplet = []
        for quatrain in range(NUM_QUATRAINS):
            for line in range(QUATRAIN_LINES):
                quatrains.append(tokenizer(raw[cursor]))
                cursor += 1
        for line in range(COUPLET_LINES):
            couplets.append(tokenizer(raw[cursor]))
            cursor += 1
    return quatrains, couplets

In [71]:
############################################################
# Class to train HMM models.
############################################################
class HMM:

    def __init__(self, num_states):

        self.D = 0 # num of unique observations
        self.L = num_states # num of hidden states

        self.token_dict = {} # map of integers to tokens

        self.A = None # transition (row: from; col: to), 0-indexed
        self.PI = None # initial state distribution, 0-indexed
        self.O = None # observation (row: state; col: observation), 0-indexed

    def train(self, data, epsilon=0.001, scaling=True):
        X = self.registerObs(data)

        L = self.L
        D = self.D

        # Initialize Matrices
        self.A = self.normalize(np.random.rand(L, L))
        self.PI = self.normalize(np.random.rand(L))
        self.O = self.normalize(np.random.rand(L, D))

        norm_arr = []
        iterations = 0

        while (True):
            iterations += 1
            # E Step
            gamma_arr, xi_arr = self.computeMarginals(X, scaling)

            # M step (Computes marginals + Updates)
            change_norm = self.update(X, gamma_arr, xi_arr)
            print change_norm
            norm_arr.append(change_norm)

            # Stopping Condition
            if len(norm_arr) > 1 and norm_arr[-1] / norm_arr[0] < epsilon:
                print "iterations:", iterations
                break

        # print self.PI
        # print self.A
        # print self.O

        return (self.token_dict, self.PI, self.A, self.O)

    """ Registers observations as integers and returns data transformed into
    integers. """
    def registerObs(self, data):
        # Reset Variables
        self.D = 0
        self.token_dict = {}

        X = [] # data transformed into integers corresponding to tokens
        for seq in data:
            X_i = [] # this sequence transformed into integers
            for token in seq:
                if token not in self.token_dict:
                    self.token_dict[token] = self.D
                    self.D += 1
                X_i.append(self.token_dict[token])
            X.append(X_i)
        return X

    """ Makes all rows add up to 1 """
    @staticmethod
    def normalize(matrix):
        if len(matrix.shape) == 1:
            return matrix / matrix.sum()
        sums = matrix.sum(axis=1)
        return matrix / sums.reshape(sums.shape[0], 1)

    def forwardBackward(self, seq, scaling=True):
        """ This function computes alpha and beta values for a sequence
            using the Forward-Backward algorithm.
        """
        M = len(seq) # length of given sequence
        L = self.L # num of states

        alphas = np.zeros((M, L)) # row: position; col: state
        betas = np.zeros((M, L)) # row: position; col: state

        # FORWARD ALGORITHM
        for i in range(M): # For each observation
            for s in range(L): # For each state
                # Base case
                if i == 0:
                    alphas[i, s] = self.O[s, seq[0]] * self.PI[s]
                else:
                    sum = 0
                    # For each previous state
                    for prev in range(L):
                        sum += alphas[i-1, prev] * self.A[prev, s]
                    alphas[i, s] = sum * self.O[s, seq[i]]
            # Scaling
            if scaling:
                scale = np.sum(alphas[i])
                alphas[i] = alphas[i] / scale

        # BACKWARD ALGORITHM
        for i in reversed(range(M)): # For each observation
            for s in range(L): # For each state
                # Base case
                if i == M-1:
                    betas[i, s] = 1
                else:
                    # For each next state
                    for next in range(L):
                        betas[i, s] += betas[i+1, next] * \
                                       self.A[s, next] * self.O[next, seq[i+1]]
            # Scaling
            if scaling:
                scale = np.sum(betas[i])
                betas[i] = betas[i] / scale

        return (alphas, betas)

    def computeMarginals(self, X, scaling=True):
        # Calculate alphas and betas for all sequences
        L = self.L
        alphas_arr = []
        betas_arr = []
        for seq in X:
            alphas, betas = self.forwardBackward(seq, scaling)
            alphas_arr.append(alphas)
            betas_arr.append(betas)
        # P(y_i = z)
        gamma_arr = [] # Indexed by: # Sequence, Position, State

        # P(y_i = prev, y_i+1 = next)
        xi_arr = [] # Indexed by: # Sequence, Position of Prev, Prev, Next

        # Compute Gammas
        for j in range(len(X)): # iterate over all sequences
            seq_len = len(X[j])
            alphas = alphas_arr[j]
            betas = betas_arr[j]

            # gammas for this sequence
            gamma = np.zeros((seq_len, L))

            for i in range(seq_len):
                for state in range(L):
                    # just numerator
                    gamma[i, state] = alphas[i, state] * betas[i, state]
                # divide by denominator
                gamma[i] = gamma[i] / gamma[i].sum()

            gamma_arr.append(gamma)

        # Compute Xi's
        for j in range(len(X)): # iterate over all sequences
            seq = X[j]
            seq_len = len(seq)
            alphas = alphas_arr[j]
            betas = betas_arr[j]

            # xi's for this sequence
            xi = np.zeros((seq_len-1, L, L))

            for i in range(seq_len-1):
                for prev in range(L):
                    for next in range(L):
                        # just numerator
                        xi[i, prev, next] = alphas[i, prev] * \
                                            self.O[next, seq[i+1]] * \
                                            self.A[prev, next] * \
                                            betas[i+1, next]
                # divide by denominator
                xi[i] = xi[i] / xi[i].sum()

            xi_arr.append(xi)

        return (gamma_arr, xi_arr)

    def update(self, X, gamma_arr, xi_arr):
        L = self.L # num states
        D = self.D # num unique tokens

        # new matrices
        PI = np.zeros(self.PI.shape)
        A = np.zeros(self.A.shape)
        O = np.zeros(self.O.shape)

        # update PI (initial distribution matrix)
        for state in range(L):
            prob_sum = 0
            for j in range(len(X)): # iterate over all sequences
                prob_sum += gamma_arr[j][0, state]

            PI[state] = prob_sum / len(X) # average across sequences

        # Make sure numbers add up to 1
        np.testing.assert_almost_equal(PI.sum(), 1)

        # Update A (transition matrix)
        for prev in range(L):
            for next in range(L):
                numerator = 0
                denominator = 0

                for j in range(len(X)): # iterate over all sequences
                    # for each index in seq excluding last index
                    for i in range(len(X[j])-1):

                        numerator += xi_arr[j][i, prev, next]
                        denominator += gamma_arr[j][i, prev]

                # UPDATE A_{prev, next}
                A[prev, next] = numerator / denominator

            # Make sure rows add up to 1
            np.testing.assert_almost_equal(A[prev].sum(), 1)

        # update O (emission matrix)
        for state in range(L):
            for token in range(D):
                numerator = 0
                denominator = 0

                for j in range(len(X)): # iterate over all sequences
                    for i in range(len(X[j])): # for each index in seq
                        prob = gamma_arr[j][i, state]

                        if X[j][i] == token: # indicator function
                            numerator += prob
                        denominator += prob

                O[state, token] = numerator / denominator

            # Make sure rows add up to 1
            np.testing.assert_almost_equal(O[state].sum(), 1)

        # frobenius norm of the differences between update and previous matrices
        change_norm = np.linalg.norm(self.A - A) + np.linalg.norm(self.O - O) \
                      + np.linalg.norm(self.PI - PI)

        # update matrices
        self.O = O
        self.PI = PI
        self.A = A

        return change_norm

In [83]:
############################################################
# Generate from trained models.
############################################################
# Length is number of syllables to generate. (Generally 10)
def gen_txt(trans, emiss, init, word_map, 
            syll_dict, stress_dict, length=10, 
            space_symb=' '):
    # Verify that the model is functional and setup.
    num_states = len(trans)
    num_words = len(emiss[0])
    assert (num_states == len(trans[0])), 'Transition matrix is not square.'
    assert (num_states == len(emiss)), 'Emission matrix not correct dimensions.'
    
    # Prepare to iterate for words.
    build = ''
    curr_state = np.random.choice(num_states, p=init)
    curr_length = 0
    curr_stress = 0
    
    # Build the sequence.
    while curr_length < length:
        # Select random word based on emission matrix.
        nxt_token = int(np.random.choice(num_words, p=emiss[int(curr_state)]))
        word = word_map[nxt_token].rstrip('.,?!;:()')
        print word
        
        # Check that word isn't too long and is stressed correctly.
        while (syll_dict[word] + curr_stress > length) or (stress_dict[word] != curr_stress):
                nxt_token = int(np.random.choice(num_words, p=emiss[int(curr_state)]))
                word = word_map[nxt_token].rstrip('.,?!;:()').lstrip('(')
                print word
        
        build += word_map[nxt_token] + space_symb
        curr_length += syll_dict[word]
        curr_stress = (curr_stress + syll_dict[word]) % 2
        
        # Go to next state.
        curr_state = np.random.choice(num_states, p=trans[int(curr_state)])
    return build

In [79]:
# Test training.
start_time = time.time()

q, c = shksp_quatrain_couplets_line(simple_token2)
q_hmm = HMM(10)
token_dict, PI, A, O = q_hmm.train(q)

print("--- %s seconds ---" % (time.time() - start_time))

KeyboardInterrupt: 

In [90]:
trans = q_hmm.A
emiss = q_hmm.O
init = q_hmm.PI
word_map = q_hmm.token_dict
word_map = dict((v,k) for k,v in word_map.iteritems())
syll_dict, stress_dic = load_syll_stress_dicts()
gen_txt(trans, emiss, init, word_map, 
        syll_dict, stress_dic)

is
indeed
poet's
sullied
starved
frame
trespass
much
ah
unseeing
hymns


"is, poet's sullied starved unseeing hymns "

In [91]:
save_file = open('matrices/10-state-quatrains-transition.p', 'wb')
pickle.dump(trans, save_file)
save_file.close()

save_file = open('matrices/10-state-quatrains-emission.p', 'wb')
pickle.dump(emiss, save_file)
save_file.close()

save_file = open('matrices/10-state-quatrains-wordmap.p', 'wb')
pickle.dump(word_map, save_file)
save_file.close()

save_file = open('matrices/10-state-quatrains-init.p', 'wb')
pickle.dump(init, save_file)
save_file.close()

In [35]:
# Set up pickle dictionary for syllable processing. 
# DO NOT RUN AFTER COMPLETING DICTIONARY!
import pickle
syll_dict = {'from' : 1}
save_file = open(SYLL_DICT, 'wb')
pickle.dump(syll_dict, save_file)
save_file.close()

In [10]:
from wordnik import *

# Method for assisting in creating syllables dict.
def make_syll_dict(lines):
    # Setup API connections.
    apiUrl = 'http://api.wordnik.com/v4'
    apiKey = '552b2562693245ea105020d08c904c58324d0d2b793995895'
    client = swagger.ApiClient(apiKey, apiUrl)
    wordApi = WordApi.WordApi(client)
    
    # Read in old dictionary.
    save_file = open(SYLL_DICT, 'rb')
    syll_dict = pickle.load(save_file)
    save_file.close()
    
    for line in lines:
        for word in line:
            if syll_dict.get(word) == None:
                try_api = wordApi.getHyphenation(word)
                if try_api != None:
                    syll_dict[word] = len(try_api)
                else:
                    print word, ':', line
                    count = input()
                    syll_dict[word] = count
        save_file = open(SYLL_DICT, 'wb')
        pickle.dump(syll_dict, save_file)
        save_file.close()
        
    return syll_dict

# Method for assisting in creating stress dict.
# Note that 0 = unstressed
#           1 = stressed
def make_stress_dict(lines, syll_dict):
    stress_dict = {}
    for line in lines:
        curr_state = 0
        for word in line:
            stress_dict[word] = curr_state
            curr_state = (curr_state + syll_dict[word]) % 2
    return stress_dict
            
# Load syllable and stress dictionaries.
def load_syll_stress_dicts():
    syll_file = open(SYLL_DICT, 'rb')
    syll_dict = pickle.load(syll_file)
    syll_file.close()
    
    stress_file = open(STRESS_DICT, 'rb')
    stress_dict = pickle.load(stress_file)
    stress_file.close()
    
    return syll_dict, stress_dict

In [27]:
all_lines = shksp_per_line(simple_token4)
print len(all_lines)

2114


In [19]:
make_syll_dict(all_lines)

beautys : ['that', 'thereby', 'beautys', 'rose', 'might', 'never', 'die']
2
feedst : ['feedst', 'thy', 'lights', 'flame', 'with', 'self-substantial', 'fuel']
1
worlds : ['thou', 'that', 'art', 'now', 'the', 'worlds', 'fresh', 'ornament']
1
makst : ['and', 'tender', 'churl', 'makst', 'waste', 'in', 'niggarding']
1
youths : ['thy', 'youths', 'proud', 'livery', 'so', 'gazed', 'on', 'now']
1
feelst : ['and', 'see', 'thy', 'blood', 'warm', 'when', 'thou', 'feelst', 'it', 'cold']
1
mothers : ['thou', 'art', 'thy', 'mothers', 'glass', 'and', 'she', 'in', 'thee']
2
natures : ['natures', 'bequest', 'gives', 'nothing', 'but', 'doth', 'lend']
2
th : ['which', 'used', 'lives', 'th', 'executor', 'to', 'be']
1
oer-snowed : ['beauty', 'oer-snowed', 'and', 'bareness', 'every', 'where']
2
summers : ['then', 'were', 'not', 'summers', 'distillation', 'left']
2
thats : ['thats', 'for', 'thy', 'self', 'to', 'breed', 'another', 'thee']
1
hearst : ['music', 'to', 'hear', 'why', 'hearst', 'thou', 'music', 'sa

{'fawn': 1,
 'pardon': 2,
 'both': 1,
 'yellow': 2,
 'four': 1,
 'hath': 1,
 'woods': 1,
 'sleep': 1,
 "friend's": 1,
 'hanging': 2,
 'mansion': 2,
 'appetite': 3,
 'captain': 2,
 'hate': 1,
 'poorly': 2,
 'whose': 1,
 'feeding': 2,
 'effectually': 4,
 'granting': 2,
 'sweetest': 2,
 'presents': 2,
 "there's": 1,
 'whatsoever': 4,
 'under': 2,
 'lord': 1,
 'sorry': 2,
 'pride': 1,
 'sway': 1,
 'worth': 1,
 'wondrous': 2,
 'discased': 2,
 'dispense': 2,
 'hadst': 1,
 'inhearse': 2,
 "women's": 2,
 'every': 3,
 'foul': 1,
 'nourished': 2,
 "o'er-read": 2,
 'prizing': 2,
 'uttering': 3,
 'prize': 1,
 'unrest': 2,
 'graced': 1,
 'succession': 3,
 'graces': 1,
 'triumph': 2,
 'enjoy': 2,
 'charter': 2,
 'force': 1,
 'suffring': 2,
 'tired': 1,
 'awake': 2,
 'razed': 1,
 'out-going': 3,
 'tires': 1,
 'crave': 1,
 'persuade': 2,
 'lourst': 1,
 'happies': 2,
 'quill': 1,
 'even': 2,
 'beated': 2,
 'evermore': 3,
 'hide': 1,
 "ne'er": 1,
 'solemn': 2,
 'thunder': 2,
 'fingers': 2,
 'liberty': 3

In [16]:
save_file = open(SYLL_DICT, 'rb')
syll_dict = pickle.load(save_file)
save_file.close()
syll_dict['children'] = 2
print len(syll_dict)

save_file = open(SYLL_DICT, 'wb')
pickle.dump(syll_dict, save_file)
save_file.close()

save_file = open(STRESS_DICT, 'rb')
stress_dict = pickle.load(save_file)
save_file.close()
stress_dict["children"] = 0
print len(stress_dict)

save_file = open(STRESS_DICT, 'wb')
pickle.dump(stress_dict, save_file)
save_file.close()

3192
3192


In [52]:
print syll_dict

{'fawn': 1, 'pardon': 2, 'yellow': 2, 'four': 1, 'hath': 1, 'sleep': 1, "friend's": 1, 'hanging': 2, 'mansion': 2, 'appetite': 3, 'captain': 2, 'hate': 1, 'poorly': 2, 'whose': 1, 'feeding': 2, 'effectually': 5, 'granting': 2, 'sweetest': 2, 'presents': 2, 'whatsoever': 4, 'under': 2, 'lord': 1, 'sorry': 2, 'pride': 1, 'sway': 1, 'worth': 1, 'frowns': 1, 'discased': 2, 'dispense': 2, 'hadst': 1, 'inhearse': 2, "women's": 2, 'every': 3, 'foul': 1, 'nourished': 2, "o'er-read": 2, 'uttering': 3, 'prize': 1, 'sunk': 1, 'graced': 1, 'succession': 3, 'graces': 1, 'triumph': 2, 'enjoy': 2, 'charter': 2, 'force': 1, 'tired': 1, 'razed': 1, 'out-going': 3, 'tires': 1, 'crave': 1, 'persuade': 2, 'down-rased': 2, 'esteem': 2, 'quill': 1, 'even': 2, 'beated': 2, 'evermore': 3, 'hide': 1, "ne'er": 1, 'solemn': 2, 'thunder': 2, 'fingers': 2, 'liberty': 3, "gild'st": 1, 'worser': 2, 'wakened': 2, 'above': 2, 'matcheth': 2, 'new': 1, 'increasing': 1, 'ever': 2, 'told': 1, 'deemed': 1, 'sinks': 1, 'nev

In [53]:
syll_dict['effectually'] = 4
syll_dict['continual'] = 3
syll_dict['master'] = 2
syll_dict['lover'] = 2
syll_dict['gentle'] = 2
syll_dict['beloved'] = 3
syll_dict['losing'] = 2
syll_dict['everywhere'] = 3
syll_dict['rebel'] = 2
syll_dict['desert'] = 2
syll_dict['refuse'] = 2
syll_dict['render'] = 2
syll_dict['elder'] = 2
syll_dict['bower'] = 2
syll_dict['travail'] = 2
syll_dict['tender'] = 2
syll_dict['salve'] = 1
syll_dict['tyrant'] = 2
syll_dict['progress'] = 2
syll_dict['present'] = 2
syll_dict['raven'] = 2
syll_dict['record'] = 2

In [54]:
print len(syll_dict)

save_file = open(SYLL_DICT, 'wb')
pickle.dump(syll_dict, save_file)
save_file.close()

3190


In [28]:
stess_dict = make_stress_dict(all_lines, syll_dict)

In [29]:
print len(stess_dict)
save_file = open(STRESS_DICT, 'wb')
pickle.dump(stess_dict, save_file)
save_file.close()

3139


In [92]:
syll_dict['progress']

2

In [25]:
save_file = open(SYLL_DICT, 'rb')
syll_dict = pickle.load(save_file)
save_file.close()
print len(syll_dict)

3319


In [26]:
save_file = open(SYLL_DICT, 'rb')
syll_dict = pickle.load(save_file)
save_file.close()
syll_dict['children'] = 2
print len(syll_dict)

save_file = open(SYLL_DICT, 'wb')
pickle.dump(syll_dict, save_file)
save_file.close()

3319
